In [5]:
import pandas as pd
import numpy as np
import pickle as p

In [6]:
data_all = pd.read_csv('data_all.csv')

In [7]:
np.random.seed(42)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=10, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

In [8]:
item_subset

[3722, 3718, 2721, 2256, 7009, 2257, 3732, 3727, 7115, 7157]

### `/train`

In [10]:
import requests
import json

url = 'http://localhost:5000/train/'

data = data_subset.to_json()

payload = {'data': data,
           'cv_acc': True,
           'project_id': 1
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [11]:
# Model Training Result
r.json()['result']

'Success'

In [12]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict(json.loads(r.json()['cv_acc']))
cv_accuracies

,item_id,avg_sales,r2_score,mae_score,mpe_score,rmse_score
0,2256.0,57.811802,0.665704,13.002121,0.224904,16.879195
1,2257.0,18.398319,0.676997,5.008902,0.272248,6.756309
2,2721.0,26.516331,0.755080,8.929934,0.336771,13.844883
3,3718.0,252.716995,0.688018,49.301423,0.195086,64.165527
4,3722.0,367.495941,0.757563,63.048675,0.171563,86.651626
5,3727.0,296.811249,0.788040,46.225693,0.155741,61.257903
6,3732.0,127.388702,0.687063,24.492716,0.192268,32.373429
7,7009.0,2009.091797,0.820981,319.864834,0.159209,448.819233
8,7115.0,29.458530,0.553408,9.560156,0.324529,13.014482
9,7157.0,12.570630,0.477590,4.772461,0.379652,6.441280


### `/predict/`

In [13]:
# Craft mock data input, average historical prices of items
prices_input = {}
for item in item_subset:
    avg_item_price = data_subset.query('Item_ID=={}'.format(item))['Price_'].mean()
    prices_input[int(item)] = round(avg_item_price,2)
    
prices_input

{3722: 10.72,
 3718: 7.6,
 2721: 13.07,
 2256: 8.27,
 7009: 4.62,
 2257: 11.54,
 3732: 11.79,
 3727: 10.81,
 7115: 4.81,
 7157: 6.66}

In [16]:
import requests
import json

url = 'http://localhost:5000/predict/'



payload = {'prices': prices_input,
           'project_id': 1
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [17]:
# What If Prediction
r.json()['pred_q']

{'2256': 57,
 '2257': 18,
 '2721': 26,
 '3718': 291,
 '3722': 361,
 '3727': 288,
 '3732': 150,
 '7009': 1922,
 '7115': 34,
 '7157': 12}